In [ ]:
import pandas as pd
import os
import glob
import concurrent.futures
import time

directory = 'C:\\Users\\minhtriet.pham\\Desktop\\Data Invoice 20230327\\NAM 2023\\Month 2'
directory2 = 'C:\\Users\\minhtriet.pham\\Desktop\\Data Invoice 20230327\\NAM 2023\\Month 2'
cate_dir = 'C:\\Users\\minhtriet.pham\\Desktop\\DataCate.xlsx'
cate_dir2 = 'C:\\Users\\minhtriet.pham\\Desktop\\DataCate.xlsx'

# list of Excel files to read data from
if os.path.exists(os.path.join(directory, '230201 Payment Summary - 2023-01-31T145747.301.xlsx')):
    excel_files = glob.glob(os.path.join(directory, '*.xlsx'))
else:
    excel_files = glob.glob(os.path.join(directory2, '*.xlsx'))
try:
    # use cols 0, Barcode; 2, Category Name
    # df2 = pd.read_excel(cate_dir, sheet_name='Sheet1', usecols=[0,2])
    # use cols 0, Barcode; 3, SubCategory Name
    df2 = pd.read_excel(cate_dir, sheet_name='Sheet1', usecols=[0,3])
    pass
except FileNotFoundError:
    # df2 = pd.read_excel(cate_dir2, sheet_name='Sheet1', usecols=[0,2])
    # use cols 0, Barcode; 3, SubCategory Name
    df2 = pd.read_excel(cate_dir2, sheet_name='Sheet1', usecols=[0,3])
    pass

outputfile =  'dataM1.csv'
# def excel2csv
def excel2csv(inputfile, outputfile):
    
    # Check if exist file_path
    if not os.path.exists(outputfile):
        with open(outputfile, 'w') as f:
            pass

    # Read the first line of the file
    with open(outputfile, 'r') as f:
        first_line = f.readline().strip()
    # Check if the first line contains the expected column names
    has_headers = 'InvoiceID' in first_line
    df = pd.read_excel(inputfile, sheet_name='Sheet2', skiprows=2, usecols=[4,7,8,14])
    # df2 Category
    
    # merge with Category, on "Barcode"
    df = df.merge(df2, on='Barcode', how='left')
    
    # Check if the first line contains the expected column names
    df.to_csv(outputfile, mode='a', header= not has_headers, index=False)


# loop through the list of Excel files
with concurrent.futures.ThreadPoolExecutor() as executor:
    [executor.submit(excel2csv, file, outputfile) for file in excel_files]
    # for file in excel_files:
    #     executor.submit(excel2csv,file)

In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Sample data
dtypes = {'InvoiceID': str, 'Barcode': str, 'Sum of Qty': str, 'Total Exclude VAT': str, 'SubCategory Name': str}
df = pd.read_csv('C:\\Users\\Trieu Pham\\Desktop\\dataT1.csv', dtype=dtypes, error_bad_lines=False)

df['Sum of Qty'] = pd.to_numeric(df['Sum of Qty'], errors='coerce').fillna(0).astype(int)
df['Total Exclude VAT'] = pd.to_numeric(df['Total Exclude VAT'], errors='coerce').fillna(0).astype(float)
df['SubCategory Name'] = df['SubCategory Name'].astype(str)
df.info()

In [ ]:
# bill0 = df1.groupby('InvoiceID').filter(lambda x: x['Sum of Qty'].sum() == 0)

bill0 = df[df['Sum of Qty'] == 0]

bill0


In [10]:
import statistics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# filter out rows where 'Sum of Qty' or 'Total Exclude VAT' are zero
df_filtered = df[(df['Sum of Qty'] > 0) & (df['Total Exclude VAT'] > 0)]


# Calculate the IQR and filter out outliers
# Q1 = np.percentile(df_filtered['Total Exclude VAT'], 5)
# Q3 = np.percentile(df_filtered['Total Exclude VAT'], 95)
# IQR = Q3 - Q1
# upper_bound = Q3 + 1.5 * IQR
# lower_bound = 1000
# df_filtered = df_filtered[df_filtered['Total Exclude VAT'] <= upper_bound]

new_df = df_filtered.dropna().groupby('InvoiceID').agg({'Barcode': lambda x: ', '.join(x.astype(str)), 
                                      'Sum of Qty': 'sum', 
                                      'Total Exclude VAT': 'sum', 
                                      'SubCategory Name': lambda x: ', '.join(x.astype(str))})

new_df



KeyboardInterrupt: 

In [ ]:
new_df.to_csv("dataCleanM2.csv")

In [ ]:

# Group the data based on Total Exclude VAT and calculate the counts
bins = np.concatenate(([0], np.arange(10000, 100001, 5000), [np.inf]))
labels = ['<10,000'] + ['{} to {}' .format(i, i+4999) for i in range(10000, 100000, 5000)] + ['>=100,000']
df_filtered['VAT_group'] = pd.cut(df_filtered['Total Exclude VAT'], bins=bins, labels=labels)
grouped_data = df_filtered.groupby('VAT_group')['Total Exclude VAT'].agg(['count','mean'])

# Calculate the mean, mode, and median of 'Total Exclude VAT' column
basket_values = new_df['Total Exclude VAT'].tolist()

mean = statistics.mean(basket_values)
mode = statistics.mode(basket_values)
median = statistics.median(basket_values)
std = statistics.stdev(basket_values)
min = min(basket_values)
max = max(basket_values)

# Print the results
print("Mean: ", mean)
print("Mode: ", mode)
print("Median: ", median)
print("Std ",std)
print("Min: ",min)
print("Max: ",max)
      


In [ ]:
# b.      How many Quantity Items in Bill 2+

qty_item_arr = bill2['Sum of Qty'].to_list()
qty_item_arr = [int(x) for x in qty_item_arr if x != 'Sum of Qty']
sum_qty = sum(qty_item_arr)

sum_qty


In [ ]:
# c.      How much Net Sales Exclude VAT in Bill 2+
import csv

qty_net_arr = bill2['Total Exclude VAT'].to_list()
# bill2['Total Exclude VAT'].to_csv("NetSales.csv",index=False)
# qty_net_arr = [float(float(x)) for x in qty_net_arr if x != 'Total Exclude VAT']
qty_net_arr = [(int(float(x))) for x in qty_net_arr if x != 'Total Exclude VAT']

with open('NetSales1.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    for x in qty_net_arr:
        writer.writerow([x])

sum_net = sum(qty_net_arr)

len(qty_net_arr)


In [ ]:
print(f"Number unique items: {len(unique_items)}")
print(f"Summary quantity items: {sum_qty}")
print(f"Summary net Sales Exclude VAT: {sum_net}")



In [ ]:
# 2.     Calculations:
# a.      What is the Percentage between Bill 1 and Bill 2+ in Total Net Sales, Total Bills, and Total Items => This only needs one result for each by month
# b.      How much Average Net Sales on Bill 2+ => To know the Real Basket Value
# c.      How many Average Unique Items in Bill 2+ => To know more insights the Basket
# d.      How many Average Items in Bill 2+ => To know more insights into the Basket


In [ ]:
# Get Bill1
bill1 = df1[df1.groupby('InvoiceID')['Sum of Qty'].transform('size') == 1]

bill1

In [ ]:
# a. What is the Percentage between Bill 1 and Bill 2+ in Total Net Sales, Total Bills, and Total Items 

## Get bill1 Total Net Sales
qty_net_arr1 = bill1['Total Exclude VAT'].to_list()
qty_net_arr1 = [int(float(x)) for x in qty_net_arr1 if x != 'Total Exclude VAT']
sum_net1 = sum(qty_net_arr1)
## Percentage between Bill 1 and Bill 2+ in Total Net Sales
total_net_perc = sum_net1/sum_net

total_net_perc

In [ ]:
## Get bill1 Total Bills
import numpy as np
unique_bill1 = np.unique(bill1['InvoiceID'])
unique_bill2 = np.unique(bill2['InvoiceID'])
## Percentage between Bill 1 and Bill 2+ in Total Bills
total_bills_perc = len(unique_bill1)/len(unique_bill2)

total_bills_perc

In [ ]:
# Quantity items in bill1
qty_item_arr1 = bill1['Sum of Qty'].to_list()
qty_item_arr1 = [int(x) for x in qty_item_arr1 if x != 'Sum of Qty']
sum_qty1 = sum(qty_item_arr1)
## Percentage between Bill 1 and Bill 2+ in Total Items
total_qty_perc = sum_qty1/sum_qty

total_qty_perc

In [ ]:
# b.      How much Average Net Sales on Bill 2+ => To know the Real Basket Value

avg_net_sales = sum_net/len(unique_bill2)

avg_net_sales

In [ ]:
# c.      How many Average Unique Items in Bill 2+ => To know more insights the Basket
avg_unique_items = len(unique_items)/len(bill2)

avg_unique_items

In [ ]:
# d.      How many Average Items in Bill 2+ => To know more insights into the Basket

avg_item_bills = sum_qty/len(bill2.drop_duplicates(subset='InvoiceID'))

avg_item_bills